# CFD Results Post-Process and Validation

**The final step for the development of a CFD simulation for a problem of interest is to process and examine the validity of the obtained numerical results (a.k.a post-processing and validation step). This steps requires an in-depth engineering knowledge about the fundamentals of the physical problem of interest. Thus, the value of investing time in understanding the fundamental physics of the flow, once again is highlighted here. This process is done starting from general inspection on CFD simulation results and ends with more detail validation process.**

The obtained results for 2D laminar flow over a rotating cylinder with Reynolds of 100 is processed and discussed in this section. The first general rule of thumb to check the convergence of a CFD simulation is to check the residual and error values for key flow variables, such as continuity, velocity and pressure. In OpenFoam the residual values for velocity components and pressure along with error values for continuity are printed out at each time step as set in the `controlDict` dictionary file. A sample of time report is shown below for the last time step `t=150`:

```C++
Time = 150

smoothSolver:  Solving for Ux, Initial residual = 0.000151043, Final residual = 3.09096e-07, No Iterations 1
smoothSolver:  Solving for Uy, Initial residual = 0.00280328, Final residual = 3.8923e-06, No Iterations 1
GAMG:  Solving for p, Initial residual = 0.00362463, Final residual = 2.10891e-05, No Iterations 4
time step continuity errors : sum local = 3.96951e-11, global = 7.39423e-13, cumulative = -5.5588e-09
GAMG:  Solving for p, Initial residual = 0.000682323, Final residual = 8.29347e-08, No Iterations 11
time step continuity errors : sum local = 1.56155e-13, global = 4.42665e-14, cumulative = -5.55876e-09
ExecutionTime = 1042.76 s  ClockTime = 1053 s

forceCoeffs forcesCoeffs output:
    Cm    = -0.0960072
    Cd    = 1.30985
    Cl    = -0.954175
    Cl(f) = -0.573095
    Cl(r) = -0.38108

Courant Number mean: 0.0238165 max: 0.565953
```

The residual value of each variable denotes how the the numerical value of that variable is differing from the previous time step. Lower value of residual indicates the obtained numerical value of the variable of interest has converged to a specific value. In this simulation this is true for `U_x`, `U_y` and `p` that have residual values by order of at least `1e-6`. The reported `continuity errors` defines how the obtained relevant variables are satisfying the continuity equation. The global continuity error with order of `1e-13` in this problem confirms that continuity equation is being satisfied.

Once the simulation is ran in OpenFoam the user can make and plot the trend of evolution for different variables at the end of the simulations. Running the command `foamJob -s simpleFoam` before starting the iterations and then at the end running the command `foamLog log` inside the main working directory will make a history of log files for different flow field variables inside a new directory called `logs`. Changing the directory into `logs` directory and running `gnuplot` command, user can plot the obtained numerical residuals. This process was done for simulation with $\alpha=2$ and generated log files were visualized using the following commands:

```C++
gnuplot
gnuplot> set xlabel "Iterations"          #sets label of x-axis
gnuplot> set ylabel "Residuals"           #sets label of y-axis
gnuplot> set logscale y                   #sets logarithmic scale of y axis
gnuplot> plot './contLocal_0' with lines, './UxFinalRes_0' with lines, './pFinalRes_0' with lines #plots residuals
```   

<img src="./Images/1residuals.png" width="500" align="middle">
</br>
Fig. 1 The initial residual values of `U_x` and `p` with the continuity error values versus time steps.

The general decreasing trend of the residual and error values confirm that the numerical solution of the conservation/governing equations are converging to specific final values. This decreasing trend is the first necessary, but not sufficient, general criteria to confirm the convergence of the CFD simulation.

The second step is to visualize dimensionless form of the important variables within the CFD domain. This visualization step can be problem specific and should be performed over an adequate plane of choice. In OpenFoam the post-processing is recommended to be done in ParaView, which is the recommended post-processor package of OpenFoam. Running the command

```C++
paraFoam
```
In the terminal, while locating in the working directory of the simulation will simply read all the obtained numerical results and get it ready for post-processing in ParaView. Once the user ran `paraFoam` command and the software load up, Activating all the boundaries and fields and then hitting the `Apply` bottom under `Properties` window will load the numerical results.

For most of the problems contour of normalized velocity or pressure, over a horizontal or vertical plane at the center of the CFD domain, is a good start point. For the current problem, since the entire domain is two dimensional and planar, it would make the entire domain the best candidate for the post-processing plane. User should now define the normalized variable of interest that needs to be visualized. Defining variables can be done via `Filters/Data Analysis/Calculator`. This command will open a new panel under `Properties`, where users can define their variable of interest. Name the variable (i.e. `U_norm`) and enter the required formula for the normalized variable of interest (i.e. `U_x/1`) and hit `Apply`. This will calculate the normalized streamwise velocity with free stream velocity within entire CFD domain for all the time steps and visualize the results for `t=0`. Using the `play` bottom users can see the formation of laminar boundary layer and wake around and in the back of the cylinder:

<img src="./Images/2velCountor.png" width="500" align="middle">
</br>
Fig. 2 - Normalized streamwise velocity contours with the free stream velocity for flow over a rotating cylinder with Re=100 and $\alpha$=1 at `t=150`.

Fig. 2 visualizes the normalized streamwise velocity with free stream for flows with Reynolds numbers of 100. The Flow enters the domain with normalized velocity of 1. Due to the imposed no slip boundary conditions over the surface of cylinder, the velocity instantly decrease to zero on the nose of the cylinder and a laminar boundary layer starts to form and grow around the cylinder's surface. After a while because of the unsteadiness of the flow at Re=100, vortex shedding happens and flow field changes periodically around the cylinder. This is also the same pattern that happens in the case were rotating velocity of the cylinder is set to zero. But as rotational speed increases, the vortex shedding behind the cylinder gradually disappears and after a critical value of $\alpha$ the vortex shedding is completely suppressed. To be able to see the vortices in the domain, user has has to close ParaView and create vorticity fields by running `vorticity` command:

```C++
vorticity
```

This will produce the vorticity field for all time steps in the case. Now again by running `paraFoam` and loading the variables, user can choose the vorticity field from the drop down list. Fig.3 shows vorticity contours around the cylinder for rotational speeds of 0.5 and 2.

<img src="./Images/3vortContours.png" width="800" align="middle">
</br>
Fig. 3 - Vorticity contours for flow over a rotating cylinder with Re=100 and `t=150`. a) $\alpha$=0.5 and b) $\alpha$=2.

Suppression of the vortex shedding removes the low-pressure region behind the cylinder and as a result a lower drag force is predicted. On the other hand, existence of high and low velocities at top and bottom parts of the cylinder will produce low and high pressure regions respectively. This pressure difference will cause a force acting in the direction normal to the free stream velocity and therefore affects the lift coefficient. This phenomena is called the **Magnus Effect**. For more information regarding this issue, see [here](https://en.wikipedia.org/wiki/Magnus_effect).

**After performing the general post-processing steps, the user need to perform more detail oriented post-processing to confirm the validity of the CFD simulation. This stage can be a state-of-the-art. Comparing the numerical results against the theory (discussed earlier) or any other available experimental results is the final stage of post-processing. It is extremely important to calculate the correct corresponding numerical variables and compare them against the experimental or other numerical data.**

In order to validate the numerical results from these simulations the numerically predicted drag coefficients of the cylinder were compared against values from a previous numerical study [here](http://cfd.mace.manchester.ac.uk/twiki/pub/Main/FlettnerRotors/Stojkovic_PoF_2002.pdf) for different rotational speeds. The reason behind choosing the drag coefficient is that the cylinder is significantly affected by the pressure forces acting on the body of cylinder and generating a wake behind the object. The value of the drag coefficient of the cylinder is defined as:

$$ C_D~=~\frac{F_D}{\frac{1}{2} . \rho_{ref} . V_{ref} ^2 . A_{proj}}. $$

In this equation $F_D$, the total drag force due to pressure and viscous forces within the flow acting on the cylinder's surface in the streamwise direction, is normalized with the available momentum within the undisturbed incoming flow that could be exerted on the cylinder. The value of this momentum is calculated using $\rho_{ref}$ as the reference density of the flow, which is the density of the flow. $V_{ref}$ as the undisturbed streamwise flow velocity in the inlet and $A_{proj}$, which is the projected area of the cylinder surface. For a two-dimensional cylinder the projected area is equal to the diameter of the cylinder. This is the area where pressure and viscus forces are acting on the cylinder in the streamwise direction. It should be noted that the pressure and viscous forces in the perpendicular direction to streamwise direction cancel each other. Hence, the forces and areas that they act on does not have any contribution to the drag force and coefficient.

As discussed earlier in the "Simulation Setup" section user can take advantage of pre-written functions and libraries to calculate and report drag force coefficient values. The "forceCoeffs" function, added to `controlDict` dictionary file located in `\system` directory, takes "cylinder" walls as the patch and calculates the forces on in the defined directions. This was completely discussed in the previous section (Open_FOAM -> Simulation).
Following table provides a summary of comparison between the present study and literature calculated drag coefficients values:

| Dimensionless Rotational Speed #  | $C_D$ Present Study| $C_D$ D. Stojković *et al.* (2002) | Relative Error [%]|
| :--------------------------------:|:------------------:|:--------------------------------:|:-----------------:|
| 0.5        | 1.2819      | 1.2740        |0.6|
| 1          | 1.1112      | 1.1080        |0.2|
| 1.5        | 0.8358      | 0.8180        |2.1|
| 2          | 0.5415      | 0.4611        |17.4|
Table 1. Comparison between the present study and literature calculated drag coefficient for different dimensionless rotational speeds.

As shown in the table the values of the relative error percentages are below 1% for cases of rotational velocities equal and below 1. The present study agrees very well with the previous numerical study for low rotational speeds. For higher speeds the corresponding drag coefficient values start to increase. The reason is the higher unsteady behavior of the flow. As rotational speed increases the value of time step have to be small enough to capture the flow field changes. This can be done by running a time step independence test and choose the appropriate value based on the obtained results.

At this stage one can conclude that this CFD numerical simulation is validated to study unsteady laminar flow over a rotating cylinder. Therefore, these simulations can be used for similar or more complex problems, such as three dimensional laminar flow over a rotating cylinder or sphere. This can be done via correct implementation in changing the corresponding solver, models and/or boundary conditions.

## Source(s)

> Stojković, D., M. Breuer, and F. Durst. "Effect of high rotation rates on the laminar flow around a circular cylinder." Physics of Fluids (1994-present) 14.9 (2002): 3160-3178.